# Full Script to Create trigrams_data

Here is the **full MySQL script** to generate the `trigrams_data` table — structured just like your monogram one, but for **three-token (trigram) combinations**, with:

- Proper column definitions
- Defensive calculations using `LEAST()` to avoid truncation/overflow
- Aligned with your `combined_data` schema (`DECIMAL(65,6)` for money, etc.)
- `ngram` length capped at 255 (safe for most trigram strings)

---

### ✅ Full Script to Create `trigrams_data`

```sql
CREATE TABLE trigrams_data (
    ngram VARCHAR(255),
    portfolio_name VARCHAR(255),
    month INT NOT NULL,
    year INT NOT NULL,
    occurrences BIGINT NOT NULL DEFAULT 0,

    impressions INT DEFAULT 0,
    clicks INT DEFAULT 0,
    spend DECIMAL(65,6) DEFAULT 0.000000,
    sales DECIMAL(65,6) DEFAULT 0.000000,
    units INT DEFAULT 0,

    ctr DECIMAL(20,10) DEFAULT 0.0000000000,
    conversion_rate DECIMAL(30,10) DEFAULT 0.0000000000,
    acos DECIMAL(30,10) DEFAULT 0.0000000000,
    cpc DECIMAL(30,10) DEFAULT 0.0000000000,
    cpa DECIMAL(30,10) DEFAULT 0.0000000000
)
AS
WITH RECURSIVE numbers AS (
    SELECT 1 AS n
    UNION ALL
    SELECT n + 1 FROM numbers WHERE n < 10
),
expanded AS (
    SELECT
        cd.customer_search_term,
        cd.impressions,
        cd.clicks,
        cd.spend,
        cd.seven_day_total_sales AS sales,
        cd.seven_day_total_units AS units,
        cd.portfolio_name,
        cd.month,
        cd.year,
        n,
        LOWER(TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(cd.customer_search_term, ' ', n), ' ', -1))) AS token1,
        LOWER(TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(cd.customer_search_term, ' ', n+1), ' ', -1))) AS token2,
        LOWER(TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(cd.customer_search_term, ' ', n+2), ' ', -1))) AS token3
    FROM combined_data cd
    JOIN numbers ON n <= (LENGTH(cd.customer_search_term) - LENGTH(REPLACE(cd.customer_search_term, ' ', '')) - 1)
),
trigrams AS (
    SELECT
        CONCAT_WS(' ', token1, token2, token3) AS ngram,
        impressions,
        clicks,
        spend,
        sales,
        units,
        portfolio_name,
        month,
        year
    FROM expanded
    WHERE token1 IS NOT NULL AND token2 IS NOT NULL AND token3 IS NOT NULL
        AND token1 != '' AND token2 != '' AND token3 != ''
        AND CHAR_LENGTH(CONCAT_WS(' ', token1, token2, token3)) <= 255
)

SELECT
    ngram,
    portfolio_name,
    month,
    year,
    COUNT(*) AS occurrences,
    SUM(impressions) AS impressions,
    SUM(clicks) AS clicks,
    SUM(spend) AS spend,
    SUM(sales) AS sales,
    SUM(units) AS units,

    -- Safe computed metrics
    LEAST(IFNULL(SUM(clicks) / NULLIF(SUM(impressions), 0), 0), 1) AS ctr,
    LEAST(IFNULL(SUM(units) / NULLIF(SUM(clicks), 0), 0), 1000000) AS conversion_rate,
    LEAST(IFNULL(SUM(spend) / NULLIF(SUM(sales), 0), 0), 1000000) AS acos,
    LEAST(IFNULL(SUM(spend) / NULLIF(SUM(clicks), 0), 0), 1000000) AS cpc,
    LEAST(IFNULL(SUM(spend) / NULLIF(SUM(units), 0), 0), 1000000) AS cpa
FROM trigrams
GROUP BY ngram, portfolio_name, month, year;
```

---

### 🔍 Notes:
- `CONCAT_WS(' ', token1, token2, token3)` creates a readable trigram like `'red rubber bands'`
- We filter for `CHAR_LENGTH(...) <= 255` to fit into `VARCHAR(255)` safely
- All computed metrics (`ctr`, `cpc`, etc.) are capped to avoid overflow and match your schema

---

Let me know if you’d like a version that generates **monograms + bigrams + trigrams into a unified table**, or if you want **indexes** added for query performance!

# Mongram Code in SQL


Absolutely — here's the **full and fixed SQL script** for creating the `monograms_data` table, optimized for:

- Real-world token lengths
- Matching your `combined_data` schema
- Avoiding overflow and truncation errors
- Including defensive handling with `LEFT()` and `LEAST()`

---

### ✅ Final Version: Safe Monogram Data Extraction

```sql
CREATE TABLE monograms_data (
    ngram VARCHAR(100),  -- Clipped to 100 characters to avoid errors
    portfolio_name VARCHAR(255),
    month INT NOT NULL,
    year INT NOT NULL,
    occurrences BIGINT NOT NULL DEFAULT 0,

    impressions INT DEFAULT 0,
    clicks INT DEFAULT 0,
    spend DECIMAL(65,6) DEFAULT 0.000000,
    sales DECIMAL(65,6) DEFAULT 0.000000,
    units INT DEFAULT 0,

    ctr DECIMAL(10,6) DEFAULT 0.000000,
    conversion_rate DECIMAL(20,6) DEFAULT 0.000000,
    acos DECIMAL(20,6) DEFAULT 0.000000,
    cpc DECIMAL(20,6) DEFAULT 0.000000,
    cpa DECIMAL(20,6) DEFAULT 0.000000
)
AS
WITH RECURSIVE numbers AS (
    SELECT 1 AS n
    UNION ALL
    SELECT n + 1 FROM numbers WHERE n < 10
),
tokenized AS (
    SELECT
        LOWER(TRIM(SUBSTRING_INDEX(SUBSTRING_INDEX(cd.customer_search_term, ' ', n), ' ', -1))) AS token,
        cd.impressions,
        cd.clicks,
        cd.spend,
        cd.seven_day_total_sales AS sales,
        cd.seven_day_total_units AS units,
        cd.portfolio_name,
        cd.month,
        cd.year
    FROM combined_data cd
    JOIN numbers ON n <= (LENGTH(cd.customer_search_term) - LENGTH(REPLACE(cd.customer_search_term, ' ', '')) + 1)
),
monograms AS (
    SELECT *
    FROM tokenized
    WHERE token IS NOT NULL AND token != '' AND CHAR_LENGTH(token) <= 100
)

SELECT
    LEFT(token, 100) AS ngram,
    portfolio_name,
    month,
    year,
    COUNT(*) AS occurrences,
    SUM(impressions) AS impressions,
    SUM(clicks) AS clicks,
    SUM(spend) AS spend,
    SUM(sales) AS sales,
    SUM(units) AS units,

    -- Safe computed metrics
    IFNULL(SUM(clicks) / NULLIF(SUM(impressions), 0), 0) AS ctr,
    LEAST(IFNULL(SUM(units) / NULLIF(SUM(clicks), 0), 0), 1000000) AS conversion_rate,
    LEAST(IFNULL(SUM(spend) / NULLIF(SUM(sales), 0), 0), 1000000) AS acos,
    LEAST(IFNULL(SUM(spend) / NULLIF(SUM(clicks), 0), 0), 1000000) AS cpc,
    LEAST(IFNULL(SUM(spend) / NULLIF(SUM(units), 0), 0), 1000000) AS cpa
FROM monograms
GROUP BY token, portfolio_name, month, year;
```

---

### 🔍 Built-in Safeguards:
| Problem                         | Fix Used                                    |
|--------------------------------|---------------------------------------------|
| Overlong token (`Data too long`) | `LEFT(token, 100)` + `CHAR_LENGTH(token) <= 100` |
| Divide by zero or tiny values  | `NULLIF(...)`, `IFNULL(...)`, `LEAST(...)`  |
| Overly wide decimals           | Reasonable precision for spend/clicks/etc   |

---

Would you like the **bigrams** and **trigrams** versions also updated with similar protection and structure?